In [ ]:
import time
import sys
from rfsoc_qsfp_offload.overlay import Overlay

board_ip = '192.168.4.99'
client_ip = '192.168.4.1'

print("Initializing RFSoC QSFP Offload Overlay")
ol = Overlay(bitfile_name="/opt/rfsoc_qsfp_10g/boards/RFSoC4x2/rfsoc_qsfp_offload/bitstream/rfsoc_offload_10g.bit", ignore_version=True)
#ol = Overlay(ignore_version=True)
ol.cmac.mmio.write(0x107C, 0x3) # RSFEC_CONFIG_ENABLE
ol.cmac.mmio.write(0x1000, 0x7) # RSFEC_CONFIG_INDICATION_CORRECTION

# Wait for overlay to initialize
time.sleep(5)

In [ ]:
# Start 100G network
ol.cmac.start()
res = ol.netlayer.set_ip_address(board_ip, debug=True)
ol.netlayer.sockets[0] = (client_ip, 60133, 60133, True)
ol.netlayer.populate_socket_table()
print("Network confguration complete IP: %s" % (res['inet addr']))

In [ ]:
# Initialize ADC and start UDP stream
f_c = 90.1                      # MHz
ADC_TILE = 2                    # ADC Tile 226 (ADC A + B)
ADC_BLOCK = 1                   # ADC Block 1 = ADC A, Block 0 = ADC B
ADC_SAMPLE_FREQUENCY = 1024     # MSps
ADC_PLL_FREQUENCY    = 491.52   # MHz 
ADC_FC = -1*f_c                 # Tune to center frequency

# Select source
adc_source = ADC_BLOCK      
ol.adc_select(adc_source)

# Stop UDP stream if already started
ol.enable_udp(False)

# Start ADC
ol.initialise_adc(tile=ADC_TILE,
                    block=ADC_BLOCK,
                    pll_freq=ADC_PLL_FREQUENCY,
                    fs=ADC_SAMPLE_FREQUENCY,
                    fc=ADC_FC)

# Decimate by (16x)
ol.set_decimation(tile=ADC_TILE,block=ADC_BLOCK,sample_rate=64e6)

# Enable UDP stream
ol.enable_udp(True)

print("Starting UDP stream")
        

In [ ]:
# Stop packet generator
ol.enable_udp(False)